In [5]:
import os
import shutil

import pandas as pd
import tqdm
import src.data.Dataset as dt

In [6]:
FROM_PATH = os.path.abspath("./dataset/emptyNonEmptyDataset")
TO_PATH = os.path.abspath("./dataset/datasetFiltered")

ORIGINAL_DATASET = os.path.abspath("./data/raw/1000Images_binary.csv")
dataset_file_name = os.path.basename(ORIGINAL_DATASET).split("_")[0] + "_filtered.csv"

FILTERED_PATH = os.path.abspath("./data/interim")
FILTERED_DATASET = os.path.join(FILTERED_PATH, dataset_file_name)

print(f"FROM_PATH:         {FROM_PATH}")
print(f"TO_PATH:           {TO_PATH}")
print(f"ORIGINAL_DATASET:  {ORIGINAL_DATASET}")
print(f"FILTERED_PATH:     {FILTERED_PATH}")
print(f"FILTERED_DATASET:  {FILTERED_DATASET}")

FROM_PATH:         /Users/carlos/WORKSPACE/MegaClassifier/dataset/emptyNonEmptyDataset
TO_PATH:           /Users/carlos/WORKSPACE/MegaClassifier/dataset/datasetReduced
ORIGINAL_DATASET:  /Users/carlos/WORKSPACE/MegaClassifier/data/raw/1000Images_binary.csv
FILTERED_PATH:     /Users/carlos/WORKSPACE/MegaClassifier/data/interim
FILTERED_DATASET:  /Users/carlos/WORKSPACE/MegaClassifier/data/interim/1000Images_reduced.csv


In [7]:
dataset = dt.load_from_csv(ORIGINAL_DATASET)
dataset

The file /Users/carlos/WORKSPACE/MegaClassifier/data/raw/1000Images_binary.csv has been successfully opened.


,file_name,label
0,\cervidae\cervidae_alumno_2_TH\29_20201023 (15...,1
1,\cervidae\CERVIDREDORFALLOWDEER_ZOO_5_6_7_9\41...,1
2,\cervidae\ciervo_PNM_2012_ME\d14266IMAG0050.JPG,1
3,\vacia\empty_caltechCameraTraps\59751691-23d2-...,0
4,\zorro\Zorro_Pepe_IREC\Zorro_Pepe_Irec_ME\IMAG...,1
...,...,...
2957,\caballo\HORSE_ZOO_1_4\2_20201117 (5162).JPG,1
2958,\caballo\HORSE_ZOO_5_6_7_9\28_20210318 (15638)...,1
2959,\cervidae\3_ciervo_simone_2\34_20201024 (5249)...,1
2960,\zorro\zorro_PNM_2013-I_ISAAC\e1641IM000112.JPG,1


In [8]:
total_images_processed = 0
total_images_copied = 0

filtered_dataset = pd.DataFrame({}, columns=dataset.columns)

for index, row in tqdm.tqdm(dataset.iterrows()):
    file_name = row["file_name"].replace("\\", "/")

    original_file = os.path.join(FROM_PATH, file_name[1:])
    
    filtered_file = file_name.lower()
    filtered_file = filtered_file.replace(" ","")
    filtered_file = filtered_file.replace("(", "_")
    filtered_file = filtered_file.replace(")", "_")
    filtered_file = filtered_file.replace("á","a").replace("é","e").replace("í","i").replace("ó","o").replace("ú","u")
    filtered_file = filtered_file.replace("ñ","n")
    
    new_row = pd.DataFrame({"file_name": [filtered_file[1:]], "label": [row["label"]]})
    filtered_dataset = pd.concat([filtered_dataset, new_row], ignore_index=True)
    
    filtered_file = os.path.join(TO_PATH, filtered_file[1:])

    os.makedirs(os.path.dirname(filtered_file), exist_ok=True)

    try:
        shutil.copyfile(original_file, filtered_file)
        total_images_copied += 1
    except FileNotFoundError:
        print(f"File not found: {original_file}")

    total_images_processed += 1

print(f"Total images processed: {total_images_processed}")
print(f"Total images copied: {total_images_copied}")

dt.dataset_to_csv(filtered_dataset, FILTERED_DATASET)

2962it [00:02, 1450.85it/s]

Total images processed: 2962
Total images copied: 2962
The dataset has been successfully saved to /Users/carlos/WORKSPACE/MegaClassifier/data/interim/1000Images_reduced.csv
